In this notebook i will extract the data from data/raw folder, two types of files load and electricity price data for different years are saved in that folder we can add more load and price files for different year for a country.

This notebook will extract the all price files in that folder and concate them and similliary for load files finally it saves one final merged file which will be consist of load and price for all years of data 

In [4]:
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

In [5]:
PROJECT_ROOT = Path.cwd().parent  # if notebook is inside notebooks/
raw_data_path = PROJECT_ROOT / "data" / "raw"
clean_data_path= PROJECT_ROOT / "data" / "clean"
final_data_path= PROJECT_ROOT / "data" / "final"



In [6]:
## concating price files
price_files = list(raw_data_path.glob("Energy_price_*.csv"))
df_list=[]
for file in price_files:
    print(f"loadinf flie: {file.name}")
    df=pd.read_csv(file)
    df_list.append(df)
energy_price=pd.concat(df_list,ignore_index=True)
energy_price.rename(columns={"MTU (CET/CEST)":"timestamp","Day-ahead Price (EUR/MWh)":"Price"},inplace=True)
energy_price=energy_price[energy_price['Sequence']=="Sequence Sequence 1"] ## Only want sequence 1 price 
energy_price=energy_price[['timestamp','Price']]
energy_price['timestamp'] = pd.to_datetime(
    energy_price['timestamp']
        .str.split(' - ')  # split start and end
        .str[0]            # take start
        .str.extract(r'(\d{2}/\d{2}/\d{4} \d{2}:\d{2}:\d{2})')[0],  # extract only datetime
    format='%d/%m/%Y %H:%M:%S')
energy_price.set_index('timestamp',inplace=True)
energy_price = energy_price[~energy_price.index.duplicated(keep='first')]
prices_df=energy_price
prices_df.shape

loadinf flie: Energy_price_22.csv
loadinf flie: Energy_price_23.csv
loadinf flie: Energy_price_24.csv


(105204, 1)

In [9]:
#checking null values in prices_df
prices_df['Price'].isna().sum()
output_file = clean_data_path / "price_clean.csv"
prices_df.to_csv(output_file)

In [ ]:
#Merging Load files in data folder
price_files = list(raw_data_path.glob("Load_*.csv"))
df_list=[]
for file in price_files:
    print(f"loadinf flie: {file.name}")
    df=pd.read_csv(file)
    df_list.append(df)
load_df=pd.concat(df_list,ignore_index=True)
load_df.rename(columns={"MTU (CET/CEST)":"timestamp","Day-ahead Total Load Forecast (MW)":"load"},inplace=True)
 
load_df=load_df[['timestamp','load']]
load_df.drop_duplicates(subset='timestamp',inplace=True)
load_df['timestamp'] = pd.to_datetime(
    load_df['timestamp']
        .str.split(' - ').str[0]
        .str.replace(r'\s*\(.*?\)', '', regex=True),
    format='%d/%m/%Y %H:%M',
    errors='coerce'
)
load_df.set_index('timestamp',inplace=True)
load_df = load_df[~load_df.index.duplicated(keep='first')]
load_df['load']=load_df['load']/1000  # converting mw into gw

loadinf flie: Load_22.csv
loadinf flie: Load_23.csv
loadinf flie: Load_24.csv


In [21]:
## Checking is there any null values
load_df['load'].isna().sum() 
#There are 200 missing values, first to check where they occur are they random or in long series
missing_load=load_df[load_df['load'].isna()]
missing_load['date'] = missing_load.index.date
missing_load['date'].value_counts().sort_index()
for day in ['2022-02-22', '2022-03-24']:
    ref_day = pd.to_datetime(day) - pd.Timedelta(days=7)
    load_df.loc[day,'load']=load_df.loc[ref_day,'load']
#still 8 random missing numbers are there
next_hour=load_df.loc['2023-10-29 01:00:00':'2023-10-29 01:45:00','load']
load_df.loc['2023-10-29 00:00:00':'2023-10-29 00:45:00','load']=next_hour.values
next_hour_2=load_df.loc['2024-10-27 01:00:00':'2024-10-27 01:45:00','load']
load_df.loc['2024-10-27 00:00:00':'2024-10-27 00:45:00','load']=next_hour.values
load_df['load'].isna().sum()
output_file = clean_data_path / "load_clean.csv"
load_df.to_csv(output_file)

print("Clean load data saved to:", output_file)

Clean load data saved to: c:\Users\lavee\Desktop\electricity price\data\clean\load_clean.csv


In [ ]:
#finally merging both load price
merged_df = load_df.join(prices_df, how="inner")
print(len(merged_df))
final_file = final_data_path / "load_price_clean.csv"
merged_df.to_csv(final_file)
print("load and price merged file saved")

105204
load and price merged file saved


False


set()

12
0


In [183]:
print(load_df[load_df.index.duplicated(keep=False)])

                         load
timestamp                    
2022-10-30 02:00:00  43175.05
2022-10-30 02:15:00  43267.11
2022-10-30 02:30:00  42659.94
2022-10-30 02:45:00  43091.01
2022-10-30 02:00:00  43085.12
2022-10-30 02:15:00  43182.88
2022-10-30 02:30:00  42603.54
2022-10-30 02:45:00  42828.53
2023-10-29 02:00:00  37179.71
2023-10-29 02:15:00  37109.63
2023-10-29 02:30:00  36949.11
2023-10-29 02:45:00  36970.68
2023-10-29 02:00:00  37179.68
2023-10-29 02:15:00  37091.29
2023-10-29 02:30:00  36918.29
2023-10-29 02:45:00  36869.87
2024-10-27 02:00:00  37480.72
2024-10-27 02:15:00  37233.36
2024-10-27 02:30:00  37123.39
2024-10-27 02:45:00  37662.25
2024-10-27 02:00:00  38023.03
2024-10-27 02:15:00  37947.60
2024-10-27 02:30:00  37924.36
2024-10-27 02:45:00  37716.56
